# Potential Energy Surface by Artificial Neural Networks 

![Figure 1](PES1.png)

Understanding the energy of large molecules plays a central role in the study of chemical and biological systems. The potential-energy surface (PES) is the most basic quantity to describe a chemical system. It determines all its properties,and once the PES is known, these properties can be extracted by different types of computer simulations. The PES is defined as a function yielding the potential-energy of an atomic configuration if the atomic coordinates are provided. 

![Figure 2](PES2.png)

In principle, information about the PES can be obtained for arbitrary atomic configurations by solving the Schrödinger equation. But, the PES should not depend on the absolute location of the atoms, only on their location relative to one another (i.e., the molecular geometry). So, a typical PES has the same dimensionality as the number of geometric degrees of freedom of the molecule (3N–6, where N = # of atoms and N > 2).

A way to model the interactions between atoms in a molecular system is by a "force field". This is a collection of equations and associated constants designed to reproduce molecular geometry and selected properties of tested structures. But most of classical force fieldswork near to the equilibrium, that means that it is not possible to model broken bonds for chemical reactions or transition states.

Even with fitted force fields the calculations have extreme computational cost, theoretical studies of these complex systems are often limited to the use of approximate methods, compromising accuracy in exchange for a speed up in the calculations.


The "ideal" potential should be:

- Accurate. Having the presicion in results similar as a high level theory level (ab initio or DFT)

- General. Useful for metals, ionic compounds, organic compunds, etc.

- “Reactive”. It means that bonds between atoms can be broken and can be made other bondings with near atoms

- Transferable. The obtained parameters of one specific atom can be used in other systems in spite of its nature  

- Fast to be evaluated. It should be cheaper than common computational methods

- Analytic derivatives must be easily available. The forces between atoms depends on analytic derivatives

Machine Learning (ML) methods have been successfully applied in a variety of applications in chemistry, including the prediction of IR spectra, reaction pathways, QM excited state energies, formation energies, atomic forces, nuclear magnetic resonance chemical shifts, and assisting in the search of novel materials. But the models are build for specific applications and therefore the parameters used for that neural networks don't have transferability.

## 1. Descriptor and NN of Behler and Parrinello

The key to transferable methods is finding a correct molecular representation that allows and improves learning in the chosen ML method. Cartesian coordinates, fractional coordinates or internal coordinates (Z-matrix) are not suitable as direct input for a neuronal network. Therefore, it is necessary a transformation onto a more suitable set of coordinates; one way is  in a **descriptor**.

A descriptor, encodes chemical identity in terms of chemical composition and atomic configuration. A descriptor to be unique if there is no pair of molecules that produces the same descriptor. The descriptor should meets all the required and desired features: 

- first principles and nuclear permutation invariance,
- translational invariance,
- rotational invariance and mirror symmetries (Euclidean symmetries),
- uniqueness, and
- differentiability.

Several representations have been developed, but true transferability and extensibility to complex chemical environments, i.e. all degrees of freedom for arbitrary organic molecules, with chemical accuracy has yet to be accomplished.

In 2007, Behler and Parrinello (BP) developed an approximate molecular representation, called symmetry functions (SF's), that take advantage of chemical locality in order to make neural network potentials (NNPs) transferable.In general, the NNPs developed in these studies are non-transferable. The idea is model the total energy as a sum of energy of each atom in the system.

![Equation 1](PES_eq1.png)

For a neural network of two layers, the energy could be modeled as: 

![Equation 2](PES_eq2.png)

where w's are the weights parameter for each node, f's the activation functions $\ G_i^\mu$ are the symmetry functions defined as: 

![Equation 3](PES_eq3.png)

The $\ f_c (R_{ij})$ , called cutoff function, define the range of the function for each atom, $\ R_{ij}$ is the distance between two atoms, and $ \ R_c$ is the cut radio for each atom, ($\ G_m^R$) is the radial function and ($\ G_m^{Amod}$) the angular function.

![Equation 4](PES_eq4.png)

where $\ R_s$ and $\eta$ are free parameter to be ajusted.

![Equation 5](PES_eq5.png)

where $\theta_{ijk}$ is the angle between three atoms, $\theta_s$ and $\zeta$ are free parameters

The symmetry functions describe the relevant enviroment of each atom. The index m is a set ot $\eta$ and $\ R_s$ parameters 

$\{m_1,m_2,m_3,...\} = \{(\eta_1, R_{s1}), (\eta_2, R_{s2}), (\eta_3, R_{s3}), ...\}$

Behler and Parrinello proposed an approach applicable to high-dimensional systems containing large numbers of atoms.This locality of the atomic interactions is known as near- sightedness in quantum chemistry. This is important because symmetry function values are invariant with respect to a rotation and translation of the system.

The neural network archithecture is single: the Cartesian coordinates $\ R_i^\alpha$ are transformed in the descriptors (symmetry functions), the information of each atom in the structure has a "flow line" and pass through a hidden neuronal network $\ S_i$ which are identical for all atoms and the output give an energy per atom, and finally all energies are added.

![Figure 3](PES3.png)

Since the introduction of this descriptor several studies have been made, however the neural network developed are non-transferable, they continues solving specific problems.

## 2. ANI

The Roitberg group in University of Florida, presented a transferable deep learning potential based on Belher and Parrinello symmetry functions that is applicable to complex and diverse molecular systems well beyond the training data set: ANAKIN-ME (Accurate NeurAl networK engINe for Molecular Energies) or ANI for short.

![Figure 4](PES4.png)

ANI is a method for developing neural network potentials (NNP's) that utilizes a modifed version of the original SFs to build single-atom atomic environment vectors (AEVs) as a molecular representation. This requires a very large data set that spans molecular conformational and configurational space, simultaneously. ANI uses an inherently parallel computational algorithm implemented in an in-house sofware package, called NeuroChem, which takes advantage of the computational power of graphics processing units (GPU) to accelerate the training, testing, and prediction of molecular total energies via an ANI potential.

The ANI method requires many training and testing datapoints, ($\ q, E_t$), where $\ q$ is some energy minimized or non-minimized molecular coordinates from a diverse set of molecules and $\ E_t$ is the single point energy calculated at a desired QM level of theory. The optimized structures get the lowest value in energy, a single value of PES. In order to get a set of data points on the PES around minima energy structure, they proposed  a method to generate structures.

In a published paper they show the procedure made for the application of this method for the database used for ANI.

![Figure 5](PES5.png)

Firstly a subset is taken from a GDB-11 database (40.3 million of molecules), which containing up 11 atoms of the atomic numbers C, N, O, and F.   

![Figure 7](PES7.png)

Molecules in GDB-11 are supplied in the form of SMILES strings, which can be converted to 3D structures using the RDKit software package. The big chemical data base encoding the molecules structure in a character chain in a format called SMILE.

![Figure 6](PES6.png)

Then, having chosen the subset, all elements are converted from SMILES stings to 3D structures adding hydrogens to fill valence orbitals. After that the molecules are optimized using a converge criteria and, with this information, normal modes displacement are generated  by calculating new little dispplacements respect to minima based on specific criteria. Finally a single point energy at the desired level of theory is calculated for modified molecule that is included in the data base.  In this way the databse include structures and energies of equilibirum conditions and some structures around that local minimum.

Using this procedure for 57 thousand molecules in a range from 10 to 24 heavy atoms up to a total of 53 aaomts, a total of 17.2 million conformations are obtained, 80% are taken for training and 20% for validation. They optimized the free parameters as weell layers and node; they used 4 layers and 128 nodes per layer with the pyramidal architecture 768:128:128:64:1 and a Gaussian activation function. Each new weight is calculated trough back-propagation from the exponential cost function.

For the aplication of ANI they made studies for isomers, conformers for retinol and a scan of PES of bond, angle bend and dihedral totations comparing ANI results and other theory level As example of PES they show a bond stretch and dihedral angle for two compounds. Some resultss about PES are shown in the next figure.

![Figure 8](PES8.png)

In general ANI make good predictions, regardless of computational cost for training and validation at least, this method is faster than classical methods (semi-empiral, molecular dynamis, DFT) for estimate energies and it is an alternative to get an accuracy results. However there are details to be considered for the modeling of PES by neural networks (NN). 
The construction of NN potentials is computationally more demanding than the development of conventional potentials because of the large number of training points needed to ensure that the NN has learned all relevant features of the PES. In this first version of ANI, structures of 8 or less C, N, O and H atoms were explored, structures with other atoms are not available to be predicted, and at this moment the model lack of generalization; the same problem of fitting parameters for force fields (reactive or not reactive).
You trainig a NN  with a range of information nad it will make predictions with structures and conditions close to the training data. in others words there is not a universal enviroment chemical NN. 
If you need to compute variations regularly for a specific system at low computational cost you can build a NN ans solve that system, but if you change a so different system  not trained with 
The application in this case was just for a determinated number of atoms in a molecule with C, H,O, N; for complex compounds with other elements would be neccesary run several molecules. The method lack of extrapolation capability to different structures; very different structures not included in the training set can not be modeled well. 


## References

- Behler J.,Perspective: Machine learning potentials for atomistic simulations J. Chem. Phys. 145, 170901 (2016)
- Behler J. and Parrinello M., Generalized Neural-Network Representation of High-Dimensional Potential-Energy Surfaces,Phys. Rev. Lett.98, 146401 (2007)
- Smith, J. S. et al. ANI-1, A data set of 20 million calculated off-equilibrium conformations for organic molecules. Sci. Data 4:170193 doi: 10.1038/sdata.2017.193 (2017)
- Smith J. S., Isayev O. and Roitberg A. E., ANI-1: an extensible neural network potential withDFT accuracy at force field computational cost, Chem. Sci. ,2017, 8,3192
- Behler J. Neural network potential-energy surfaces in chemistry: a tool for large scales simulations. Phys. Chem. Chem. Phys., 2011,13, 17930–17955


